In [1]:
from pycaret.regression import *
import pickle
import pandas as pd

In [2]:
bikerentals = pd.read_csv('https://aka.ms/bike-rentals')

job3 = load_experiment('Bike rentals experiment', data = bikerentals)

,Description,Value
0,Session id,1234
1,Target,rentals
2,Target type,Regression
3,Original data shape,"(731, 13)"
4,Transformed data shape,"(731, 13)"
5,Transformed train set shape,"(511, 13)"
6,Transformed test set shape,"(220, 13)"
7,Numeric features,12
8,Preprocess,True
9,Imputation type,simple


In [3]:
modelo = open('modelo_bike_rentals_pycaret','rb')

modelo_final = pickle.load(modelo)

modelo.close()

In [4]:
create_api(modelo_final, 'bike_rentals_api')

API successfully created. This function only creates a POST API, it doesn't run it automatically. To run your API, please run this command --> !python bike_rentals_api.py


In [5]:
# %load bike_rentals_api.py

import pandas as pd
from pycaret.regression import load_model, predict_model
from fastapi import FastAPI
import uvicorn
from pydantic import create_model

# Create the app
app = FastAPI()

# Load trained Pipeline
model = load_model("bike_rentals_api")

# Create input/output pydantic models
input_model = create_model("bike_rentals_api_input", **{'day': 15.0, 'mnth': 5.0, 'year': 2012.0, 'season': 2.0, 'holiday': 0.0, 'weekday': 2.0, 'workingday': 1.0, 'weathersit': 2.0, 'temp': 0.6116669774055481, 'atemp': 0.5764039754867554, 'hum': 0.7945830225944519, 'windspeed': 0.14739200472831726})
output_model = create_model("bike_rentals_api_output", prediction=331)


# Define predict function
@app.post("/predict", response_model=output_model)
def predict(data: input_model):
    data = pd.DataFrame([data.dict()])
    predictions = predict_model(model, data=data)
    return {"prediction": predictions["prediction_label"].iloc[0]}


if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=8000)


In [6]:
!python bike_rentals_api.py